In [1]:
%load_ext sql

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyodbc
import sqlalchemy
import sqlite3
from subprocess import check_output
import os

%sql sqlite://

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: @None'

## Aggregate functions

Often, you will want to perform some calculation on the data in a database. SQL provides a few functions, called aggregate functions, to help you out with this.

In this session; we will load database from file.csv then put it into the tables in file.sql

First of all, take a look to our dataset

In [2]:
ratings = pd.read_csv('/kaggle/input/imdb-extensive-dataset/IMDb ratings.csv')
print(ratings.info())
ratings.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81273 entries, 0 to 81272
Data columns (total 49 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   imdb_title_id              81273 non-null  object 
 1   weighted_average_vote      81273 non-null  float64
 2   total_votes                81273 non-null  int64  
 3   mean_vote                  81273 non-null  float64
 4   median_vote                81273 non-null  float64
 5   votes_10                   81273 non-null  int64  
 6   votes_9                    81273 non-null  int64  
 7   votes_8                    81273 non-null  int64  
 8   votes_7                    81273 non-null  int64  
 9   votes_6                    81273 non-null  int64  
 10  votes_5                    81273 non-null  int64  
 11  votes_4                    81273 non-null  int64  
 12  votes_3                    81273 non-null  int64  
 13  votes_2                    81273 non-null  int

,imdb_title_id,weighted_average_vote,total_votes,mean_vote,median_vote,votes_10,votes_9,votes_8,votes_7,votes_6,...,females_30age_avg_vote,females_30age_votes,females_45age_avg_vote,females_45age_votes,top1000_voters_rating,top1000_voters_votes,us_voters_rating,us_voters_votes,non_us_voters_rating,non_us_voters_votes
0,tt0000574,6.1,537,6.3,6.0,54,17,55,121,122,...,6.0,19.0,6.6,14.0,6.3,64.0,6.0,89.0,6.2,309.0
1,tt0001892,5.9,171,6.1,6.0,5,6,17,41,52,...,5.8,4.0,6.5,8.0,5.9,29.0,6.2,27.0,6.0,114.0


In [3]:
names = pd.read_csv('/kaggle/input/imdb-extensive-dataset/IMDb names.csv')
print(names.info())
names.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175715 entries, 0 to 175714
Data columns (total 20 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   imdb_name_id           175715 non-null  object 
 1   name                   175715 non-null  object 
 2   birth_name             175715 non-null  object 
 3   height                 30080 non-null   float64
 4   bio                    122832 non-null  object 
 5   birth_details          75229 non-null   object 
 6   birth_year             75111 non-null   float64
 7   date_of_birth          75102 non-null   object 
 8   place_of_birth         71122 non-null   object 
 9   death_details          26862 non-null   object 
 10  death_year             26838 non-null   float64
 11  date_of_death          26855 non-null   object 
 12  place_of_death         25036 non-null   object 
 13  reason_of_death        11972 non-null   object 
 14  spouses                175715 non-nu

,imdb_name_id,name,birth_name,height,bio,birth_details,birth_year,date_of_birth,place_of_birth,death_details,death_year,date_of_death,place_of_death,reason_of_death,spouses,divorces,spouses_with_children,children,primary_profession,known_for_titles
0,nm0000001,Fred Astaire,Frederic Austerlitz Jr.,177.0,"Fred Astaire was born in Omaha, Nebraska, to J...","May 10, 1899 in Omaha, Nebraska, USA",1899.0,1899-05-10,"Omaha, Nebraska, USA","June 22, 1987 in Los Angeles, California, USA ...",1987.0,1987-06-22,"Los Angeles, California, USA",pneumonia,2,0,1,2,"soundtrack,actor,miscellaneous","tt0050419,tt0053137,tt0072308,tt0043044"
1,nm0000002,Lauren Bacall,Betty Joan Perske,174.0,Lauren Bacall was born Betty Joan Perske on Se...,"September 16, 1924 in The Bronx, New York City...",1924.0,1924-09-16,"The Bronx, New York City, New York, USA","August 12, 2014 in New York City, New York, US...",2014.0,2014-08-12,"New York City, New York, USA",stroke,2,1,2,3,"actress,soundtrack","tt0037382,tt0038355,tt0117057,tt0071877"


In [4]:
movies = pd.read_csv('/kaggle/input/imdb-extensive-dataset/IMDb movies.csv')
print(movies.info())
movies.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81273 entries, 0 to 81272
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   imdb_title_id          81273 non-null  object 
 1   title                  81273 non-null  object 
 2   original_title         81273 non-null  object 
 3   year                   81273 non-null  int64  
 4   date_published         81273 non-null  object 
 5   genre                  81273 non-null  object 
 6   duration               81273 non-null  int64  
 7   country                81234 non-null  object 
 8   language               80518 non-null  object 
 9   director               81200 non-null  object 
 10  writer                 79780 non-null  object 
 11  production_company     76948 non-null  object 
 12  actors                 81207 non-null  object 
 13  description            78843 non-null  object 
 14  avg_vote               81273 non-null  float64
 15  vo

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,NaN,Charles Tait,...,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,537,$ 2250,NaN,NaN,NaN,7.0,7.0
1,tt0001892,Den sorte drøm,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",NaN,Urban Gad,...,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.9,171,NaN,NaN,NaN,NaN,4.0,2.0


### Assign to table

In [5]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:////IMDB', echo=False)
movies.to_sql('movies', con = engine)
names.to_sql('names', con = engine)
ratings.to_sql('ratings', con = engine)

# PRACTICEs

## 0. WARMING-UPs

Using `%sql` and `SELECT, FROM ,...` keyword to extract table

In [6]:
%sql PERSIST names
%sql SELECT name, birth_year, date_of_birth FROM names WHERE birth_year BETWEEN 1900 AND 1910 LIMIT 6

 * sqlite://
 * sqlite://
Done.


name,birth_year,date_of_birth
Gary Cooper,1901.0,1901-05-07
Bette Davis,1908.0,1908-04-05
Marlene Dietrich,1901.0,1901-12-27
Henry Fonda,1905.0,1905-05-16
Clark Gable,1901.0,1901-02-01
John Gielgud,1904.0,1904-04-14


But; I prefer using `pd.read_sql` after using the keywords `SELECT, FROM, WHERE, etc.`

In [7]:
pd.read_sql(
    """
        SELECT name, birth_year, date_of_birth
        FROM names
        WHERE birth_year BETWEEN 1900 AND 1910
        LIMIT 9
        """, con = engine)

,name,birth_year,date_of_birth
0,Gary Cooper,1901.0,1901-05-07
1,Bette Davis,1908.0,1908-04-05
2,Marlene Dietrich,1901.0,1901-12-27
3,Henry Fonda,1905.0,1905-05-16
4,Clark Gable,1901.0,1901-02-01
5,John Gielgud,1904.0,1904-04-14
6,Cary Grant,1904.0,1904-01-18
7,Katharine Hepburn,1907.0,1907-05-12
8,Akira Kurosawa,1910.0,1910-03-23


## 1. AVERAGE, SUM, MAX & MIN() functions

### 1.1. SUM()
Return the total values of a specific column. 

### 1.2. AVG()
Return the average values of a specific column.
### 1.3. MAX() and MIN()
Return the highest and lowest value of a specific column.
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~`

For example in table movies; use: 

**SUM()** to get the `total duration of all films.`

**AVG()** to explore average duration of all films.

**MAX() & MIN()** to get the durations of the longest and the shortest film.

In [8]:
pd.read_sql(
    """ 
    SELECT SUM(duration),
           AVG(duration),
           MAX(duration),
           MIN(duration)
    FROM movies""", con = engine)

,SUM(duration),AVG(duration),MAX(duration),MIN(duration)
0,8173299,100.565981,3360,40


Likewise, if we want to know the number of review from users

In [9]:
pd.read_sql(
    """ 
    SELECT SUM(reviews_from_users),
           AVG(reviews_from_users),
           MAX(reviews_from_users),
           MIN(reviews_from_users)
    FROM movies""", con = engine)

,SUM(reviews_from_users),AVG(reviews_from_users),MAX(reviews_from_users),MIN(reviews_from_users)
0,3246312.0,43.753194,8302.0,1.0


## 2. Combining aggregate functions with WHERE

Aggregate functions can be combined with the WHERE clause to gain further insights from your data.

For example; extract 4 subsets from `movies` database which satisfies:

(i) Contain the total numbers of reviews from users by all films made in 2000 or later.

In [10]:
pd.read_sql(
    """ 
    SELECT SUM(reviews_from_users)
    FROM movies
    WHERE year >= 2000
    """, con = engine)

,SUM(reviews_from_users)
0,2095359.0


(ii) Contain the average durations by all films whose title start with the letter 'T'

In [11]:
pd.read_sql(
    """ 
    SELECT AVG(duration)
    FROM movies
    WHERE title LIKE 'T%'
    """, con = engine)

,AVG(duration)
0,97.65914


(iii) Contain the film worst review from user in 1999.

In [12]:
pd.read_sql(
    """ 
    SELECT MIN(reviews_from_users)
    FROM movies
    WHERE year = 1999
    """, con = engine)

,MIN(reviews_from_users)
0,1.0


(iv) Contain the film best review from user between 2000 and 2012, inclusive.

In [13]:
pd.read_sql(
    """ 
    SELECT MAX(reviews_from_users)
    FROM movies
    WHERE year BETWEEN 2000 AND 2012
    """, con = engine)

,MAX(reviews_from_users)
0,6344.0


## 3. ANOTHER ARITHMETIC (+ - * /)

First of all, review the **extract from non-missing_values** by using `IS NOT NULL` then **finding exactly the ages of actors** by 

                    age = death_year - birth_year
                    
Noting that the values in both columns `death_year` and `birth_year` must be `non-missing`!!

In [14]:
pd.read_sql(
    """ 
    SELECT name, birth_year, death_year,
           reason_of_death, (death_year - birth_year) AS age
    FROM names
    WHERE birth_year IS NOT NULL
    AND death_year IS NOT NULL
    """, con = engine)

,name,birth_year,death_year,reason_of_death,age
0,Fred Astaire,1899.0,1987.0,pneumonia,88.0
1,Lauren Bacall,1924.0,2014.0,stroke,90.0
2,John Belushi,1949.0,1982.0,acute cocaine and heroin intoxication,33.0
3,Ingmar Bergman,1918.0,2007.0,None,89.0
4,Ingrid Bergman,1915.0,1982.0,lymphoma complications following a breast canc...,67.0
...,...,...,...,...,...
26221,Agathe von Trapp,1913.0,2010.0,congestive heart failure,97.0
26222,Karlene Faith,1938.0,2017.0,aortic aneurysm,79.0
26223,Phyllis Spreadbury,1933.0,2002.0,coronary thrombosis,69.0
26224,Noelle Balfour,1984.0,2017.0,complications from injuries sustained in a car...,33.0


**Next; get the percentage of people who are no longer alive (based on columns death_year).**

Alias the result as percentage_dead. Remember to use 100.0 and not 100! 

Remember that the preceding command using `IS NOT NULL` to simplifier the output table; in this query it is **stupid** when using `IS NOT NULL` again (because the function COUNT() didnt' work on the missing values)

In [17]:
pd.read_sql(
    """
    SELECT 
        COUNT(death_year) AS total_deaths,
        COUNT(*) AS total_observations,
        ROUND(COUNT(death_year) * 100.0 / COUNT(*), 3) AS percentage_dead
    FROM names
    """, con = engine)

,total_deaths,total_observations,percentage_dead
0,26838,175715,15.274


**Explore the times between the newest and oldest films**

In [16]:
pd.read_sql("""
        SELECT 
            MIN(year) AS oldest_film,
            MAX(year) AS newest_film,
            MAX(year) - MIN(year) AS difference,
            (MAX(year) - MIN(year)) / 10.0 AS number_of_decades,
            (MAX(year) - MIN(year)) / 10.0 AS number_of_century
        FROM movies
        """, con = engine)

,oldest_film,newest_film,difference,number_of_decades,number_of_century
0,1906,2019,113,11.3,11.3


Likewise, if we consider the percentage & number of alives

In [18]:
pd.read_sql(
    """
    SELECT 
        (COUNT(*) - COUNT(birth_year)) AS total_alives,
        COUNT(*) AS total_observations,
        ROUND(100 - (COUNT(death_year) * 100.0 / COUNT(*)), 3) AS percentage_alive
    FROM names
    """, con = engine)

,total_alives,total_observations,percentage_alive
0,100604,175715,84.726
